In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
bs = load_boston()

In [ ]:
bs.keys()

In [ ]:
print(bs.DESCR)

In [ ]:
bs.data.shape

In [ ]:
type(bs)

In [ ]:
bs.data

In [ ]:
boston = pd.DataFrame(bs.data, columns = bs.feature_names)
boston['MEDV'] = bs.target

In [ ]:
boston

# Data Exploration : Correlation

In [ ]:
f, ax = plt.subplots(1,3, figsize = (20,5))
ax[0].scatter(boston.RM , boston.MEDV)
ax[0].set_xlabel('RM')
ax[0].set_ylabel('MEDV')
ax[0].set_title('RM', size = 22)
ax[1].scatter(boston.AGE , boston.MEDV)
ax[1].set_xlabel('AGE')
ax[1].set_ylabel('MEDV')
ax[1].set_title('AGE', size = 22)
ax[2].scatter(boston.LSTAT , boston.MEDV)
ax[2].set_xlabel('LSTAT')
ax[2].set_ylabel('MEDV')
ax[2].set_title('LSTAT', size = 22)
plt.show()

In [ ]:
boston.drop(columns= 'CHAS').corr()

## Heat Map : Correlation Matrix

In [ ]:
m = np.ones_like(boston.drop(columns = 'CHAS').corr())
m[np.tril_indices_from(m)]=0

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(boston.drop(columns = 'CHAS').corr(), annot= True, annot_kws= {'size' : 12},
           cmap = 'Blues', fmt = '.2f', linewidths= 2, linecolor='white', mask = m,vmin=-1)
plt.show();

# 2. Simple Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split as split
train, test = split(boston, test_size = 0.20, random_state = 12)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train

In [ ]:
test

### Model Development

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
??LinearRegression

In [ ]:
# create a model object
lm = LinearRegression()

In [ ]:
# fit the model 
X = train[['RM']] # always needs to be dataframe
y= train.MEDV # always a Series

In [ ]:
plt.figure(figsize = (10,5))
plt.scatter(train.RM, train.MEDV)
plt.show();

In [ ]:
lm.fit(X, y)

In [ ]:
# Parameters
lm.coef_ # beta value(s)

In [ ]:
lm.intercept_ # intercept

In [ ]:
y_cap = lm.predict(train[['RM']])
plt.figure(figsize = (10,5))
plt.scatter(x=train.RM, y=train.MEDV)
plt.plot(train.RM, y_cap,color = 'red')
plt.show();

In [ ]:
lm.score(X, y) # R squared value

In [ ]:
from statsmodels.formula.api import ols # ordinary least square 

In [ ]:
mod = ols(formula='MEDV ~ RM', data = train)

In [ ]:
lm_fit = mod.fit()

In [ ]:
lm_fit.summary()

## 3. Multiple Linear Regression

In [ ]:
ft = ' + '.join(train.drop(columns = 'MEDV'))
formula = 'MEDV ~ ' + ft
formula

In [ ]:
lm_fit_multi = ols(formula, data = train).fit()

In [ ]:
lm_fit_multi.summary()

### Plotting the coefficients

In [ ]:
plt.figure(figsize=(10, 4), dpi=120, facecolor='w', edgecolor='b')
x = lm_fit_multi.params.index
y = lm_fit_multi.params
plt.bar( x, y )
plt.xlabel( "Variables")
plt.ylabel('Coefficients')
plt.title('Coefficient plot');

### RMSE [Root Mean Squared Error]

In [ ]:
from sklearn.metrics import mean_squared_error as mse
fitted = lm_fit_multi.fittedvalues
resid = lm_fit_multi.resid

In [ ]:
fitted=lm_fit_multi.predict(train)
fitted[:5]

In [ ]:
np.mean((train.MEDV-fitted)**2)

In [ ]:
np.mean((train.MEDV-fitted)**2)**.5

In [ ]:
### Training Data
## 1. MSE
print("MSE  - ",mse(y_true = train.MEDV, y_pred = fitted, squared = True).round(1))
## 2. RMSE
print("RSME - " ,mse(y_true = train.MEDV, y_pred = fitted, squared = False).round(1))

In [ ]:
# predictions & validation on Test Data set
test_predicted=lm_fit_multi.predict(test)

## 1. MSE
print("MSE  - ",mse(y_true = test.MEDV, y_pred = test_predicted, squared = True).round(1))
## 2. RMSE
print("RSME - " ,mse(y_true = test.MEDV, y_pred = test_predicted, squared = False).round(1))

# ANOVA : Overall Feasibilty of Regression Model
      - H0 : all the beta values are non significant
      -    : none of varaibles are significant
      -    : all the regression terms are non significant 
      -    : hence regression is not at all possible

# T - Test : Variable Significance 

* P - Value is uded to reject the H0
* H0 : the corresponding beta value is non significant hence the corresponding variable is not significant
* for whichever variable P- Value is high [>0.5] - We fail to reject the null, hence varaiable is not significcant 

# AIC - Akeike information Criterion = 2p - 2 ln L
# BIC - Baysean Information Criterion = pln n - 2ln L
    - based on Information theory : information loss
    - lower value of AIC & BIC
    - Relative 
    - p in the equation : no. of factors / x variables 

# <center> Linear Regression - Deep Dive 

## Assumptions of Linear Regression

* Linear Relation : The relationship b/w Dependent & Independent is supposed to be linear. Dependent varaibale is a linear function of independent variable & error term
* Errors [Residuals] are Normally Distributed i.e. Regression Model is Robust
* Homoscedasticity : Equal variance in the errors across dataset
* Multicollinearity : Independence of the independent varaibles 

In [ ]:
# Arranging and calculating the Residuals
residuals = pd.DataFrame({
    'fitted values' : train.MEDV,
    'predicted values' : fitted,
})

residuals['residuals'] = residuals['fitted values'] - residuals['predicted values']
residuals.head()

### A. Linearity Assumption

In [ ]:
# ------------ Already validated Before Model Building --------
y_cap = lm.predict(train[['RM']])
plt.figure(figsize = (10,5))
plt.scatter(train.RM, train.MEDV)
plt.plot(train.RM, y_cap,color = 'red')
plt.show();

### B. Normality of Errors

**Residual Analysis  :**
* Predicted / estimated = y_cap
* Residuals = y - y_cap 

In [ ]:
import scipy.stats as stats

In [ ]:
# Histogram for distribution
plt.figure(figsize=(8, 4), dpi=120, facecolor='w', edgecolor='b')
plt.hist(residuals.residuals, bins = 150)
plt.xlabel('Error')
plt.ylabel('Frequency')
plt.title('Distribution of Error Terms')
plt.show()

According to the Histogram, the distribution of error is not perfectly normal and there are some outliers on the Higher end of the errors.

### QQ-Plot (Is the data Normally Distributed?)

In [ ]:
# importing the QQ-plot from the from the statsmodels
from statsmodels.graphics.gofplots import qqplot

## Plotting the QQ plot
fig, ax = plt.subplots(figsize=(5,5) , dpi = 120)
qqplot(residuals.residuals, line = 's' , ax = ax)
plt.ylabel('Residual Quantiles')
plt.xlabel('Ideal Scaled Quantiles')
plt.title('Checking distribution of Residual Errors')
plt.show()

The QQ-plot clearly verifies our findings from the the histogram of the residuals, the data is not exactly normal in nature, and there are some outliers on the higher end of the Residues.

### C. Homoscedasticity 
* Scale - Location plot 
* Sqrt of standardised residuals against fitted values

In [ ]:
plt.figure(figsize=(8, 4), dpi=120, facecolor='w', edgecolor='b')
f = range(0,len(train))
k = [0 for i in range(0,len(train))]
plt.scatter( f, residuals.residuals[:], label = 'Residuals')
plt.plot( f, k , color = 'red', label = 'Mean' )
plt.xlabel('fitted points ')
plt.ylabel('residuals')
plt.title('Residual plot')
#plt.ylim(-4000, 4000)
plt.legend();


### D. Multicollinearity

* VIF - Variance Inflation Factor
* VIF(x) = 1 / (1 - R-Squared(X))
* R-Squared(X) is the R-Squared value coming from regression model for x i.e. taking X as dependent and all other variables as the independent variables

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [ ]:
# vif requires a data with intercept column
x_data = train.drop(columns= 'MEDV')
x_data['Intercept'] = 1
x_data

In [ ]:
?vif

In [ ]:
vif(x_data.values, 1)

In [ ]:
VIF = pd.DataFrame()
VIF['Independent_Variables'] = x_data.columns
VIF['VIF'] = [vif(x_data.values, i) for i in range(x_data.shape[1])]
VIF = VIF.set_index('Independent_Variables').drop(index = 'Intercept').T

In [ ]:
VIF

In [ ]:
x_data=x_data.drop('Intercept',axis=1)

In [ ]:
x_data

In [ ]:
x_data.columns

In [ ]:
vif_d = {}
for var in x_data.columns:
    form =  var + '~ '+ ' + '.join(x_data.drop(columns = [var])) 
    print(form)
    mod = ols(form, data = x_data).fit()
    vif_d[var] = 1/ (1-mod.rsquared)

In [ ]:
pd.DataFrame(vif_d, index = ['VIF'])

In [ ]:
# identify the variables Using VIF 
drop_cols = ['MEDV', 'TAX']
x_data = train.drop(columns= drop_cols)
x_data['intercept'] = 1
VIF = pd.DataFrame()
VIF['variables'] = x_data.columns
VIF['VIF'] = [vif(x_data.values, i) for i in range(x_data.shape[1])]
VIF = VIF.set_index('variables').drop(index = 'intercept').T
VIF

In [ ]:
# identify the variables Using VIF 
drop_cols = ['MEDV', 'TAX','NOX']
x_data = train.drop(columns= drop_cols)
x_data['intercept'] = 1
VIF = pd.DataFrame()
VIF['variables'] = x_data.columns
VIF['VIF'] = [vif(x_data.values, i) for i in range(x_data.shape[1])]
VIF = VIF.set_index('variables').drop(index = 'intercept').T
VIF

In [ ]:
# Model Dev - Post MultiCollinearity Treatment
drop_cols = ['MEDV', 'TAX','NOX']
form =   'MEDV ~ '+ ' + '.join(train.drop(columns = drop_cols))
print(form)
mod_lm =ols(form, data = train).fit()
mod_lm.summary()

In [ ]:
# Model Dev - Iterations
drop_cols = ['MEDV', 'TAX','NOX','AGE']
form =   'MEDV ~ '+ ' + '.join(train.drop(columns = drop_cols))
print(form)
mod_lm =ols(form, data = train).fit()
mod_lm.summary()

In [ ]:
# Model Dev - Iterations
drop_cols = ['MEDV', 'TAX','NOX','AGE','RAD']
form =   'MEDV ~ '+ ' + '.join(train.drop(columns = drop_cols))
print(form)
mod_lm =ols(form, data = train).fit()
mod_lm.summary()

### Final Model

In [ ]:
form

In [ ]:
# final model
mod_lm = ols('MEDV ~ CRIM + ZN + INDUS + CHAS + RM + DIS + PTRATIO + B + LSTAT', data = train).fit()
mod_lm.summary()

In [ ]:
train

## Model Interpretability [Bonus]

So far we have simply been predicting the values using the linear regression, But in order to Interpret the model, the normalising of the data is essential.

In [ ]:
from sklearn.linear_model import LinearRegression

# create a model object [instance of Linear Regresssion]
lm = LinearRegression(normalize = True)

# Fitting the model
lm.fit(train.drop(['MEDV', 'TAX','NOX','AGE','RAD'],axis=1), train["MEDV"])

In [ ]:
lm.score(train.drop(['MEDV', 'TAX','NOX','AGE','RAD'],axis=1), train["MEDV"])

In [ ]:
fitted = lm.predict(train.drop(['MEDV', 'TAX','NOX','AGE','RAD'],axis=1))
#resid = lm.predict(X)

In [ ]:
### Training Data
## 1. MSE
print("MSE  - ",mse(y_true = train.MEDV, y_pred = fitted, squared = True).round(1))
## 2. RMSE
print("RSME - " ,mse(y_true = train.MEDV, y_pred = fitted, squared = False).round(1))

In [ ]:
plt.figure(figsize=(8, 6), dpi=120, facecolor='w', edgecolor='b')
x = train.drop(['MEDV', 'TAX','NOX','AGE','RAD'],axis=1).columns
y = lm.coef_
plt.bar( x, y )
plt.xlabel( "Variables")
plt.ylabel('Coefficients')
plt.title('Normalized Coefficient plot');